In [42]:
## import packages 

from bs4 import BeautifulSoup as Soup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pulp
import re

In [43]:
## Input point values URL (very specific to Draft Sharks - the thing with BS is it's kind of unique to each page)

ffc_response = requests.get('https://www.draftsharks.com/adp/half/all?AdpForm%5BleagueSize%5D=12')

In [44]:
## parse data with Soup (creates nested Tags) 

adp_soup = Soup(ffc_response.text) 
tables = adp_soup.find_all('table')

## since the table is in a list with 'tables', you have to access the actual element in the list

adp_table = tables[0]


In [45]:
## the table is still nested, so run find_all to return the 'tr' (table rows) 

rows = adp_table.find_all('tr')

In [46]:
## now parse the entire table down to list form

temp = [x.find_all('td') for x in rows]

In [47]:
## iterate through rows of list #temp and then iterate through THOSE values with get_text

clean = [([str(y.get_text()) for y in x]) for x in temp]


In [48]:
## convert final list of lists to a data frame 

df = pd.DataFrame(clean)
df.head()

,0,1,2,3,4
0,None,None,None,None,None
1,1. Justin JeffersonWR,1.1 Show Trend »,13,207,284.20
2,2. Christian McCaffreyRB,1.2 Show Trend »,9,226,238.35
3,3. Ja'Marr ChaseWR,1.3 Show Trend »,7,200,266.85
4,4. Tyreek HillWR,1.4 Show Trend »,10,187,262.15


In [49]:
## Drop first row 
df.drop(0, axis = 0, inplace = True)

## split first column for name/position 

df['position'] = df[0].str[-2:]

## Name your columns

df.columns = ['name', 'avg_position', 'bye_week', 'consensus_points', 'ds_points', 'position']

df['name'] = df['name'].str[:-2]

## remove the numeric values from player names and then remove the '.' from the beginning 

df['name'] = df['name'].str.replace('\d+', '')
df['name'] = df['name'].str[2:]

## drop ADP column since we don't need it for an auction draft
df.drop('avg_position', axis = 1, inplace = True)
df.drop('bye_week', axis = 1, inplace = True)
## Everything is in string form, so we need to make them the right data type

float_cols =['ds_points']
int_cols =['consensus_points'] 

df[float_cols] = df[float_cols].astype(float)
df[int_cols] = df[int_cols].astype(int)

In [50]:
## create positional tables for use 
df_wr = df.loc[df['position'] == 'WR']
df_rb = df.loc[df['position'] == 'RB']
df_qb = df.loc[df['position'] == 'QB']
df_te = df.loc[df['position'] == 'TE']

In [51]:
## sort values and reset index
df_wr.sort_values(['consensus_points'], ascending = False, inplace = True)
df_wr.reset_index(inplace = True, drop = True)
df_rb.sort_values(['consensus_points'], ascending = False, inplace = True)
df_rb.reset_index(inplace = True, drop = True)
df_qb.sort_values(['consensus_points'], ascending = False, inplace = True)
df_qb.reset_index(inplace = True, drop = True)
df_te.sort_values(['consensus_points'], ascending = False, inplace = True)
df_te.reset_index(inplace = True, drop = True)

/Users/cameronfryzel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/cameronfryzel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/cameronfryzel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
## Set budgets 
wr_rb_budget = 69
qb_te_budget = 28


In [53]:
## Set # of Starters for the league
wr_rb_starters = 30 ## 2.5 starters per position (2 starters WR/RB(4) + 1 flex)
qb_te_starters = 12


In [54]:
## Set # of bench players for the league
wr_rb_bench = 24 ## 2 per team/position. 
qb_te_bench = 6 ## not enough quality starters for everyone to carry 1 backup (planning for adds/drops on byes)

In [55]:
## calculate replacement player value (avg of the bench players (aka $1 players/replaceable) 
## UPDATE: 2023 Post Draft Analysis - Everyone goes hard early. Avg of 4.5 players/team with over $10 bids. 
    ## Adjusting replacement player value to 90th percentile of bench players instead of 20th. Won't make a huge
    ## difference but worth adjusting 
wr_replacement_value = df_wr['consensus_points'].iloc[wr_rb_starters:(wr_rb_starters + wr_rb_bench)].quantile(.9)
rb_replacement_value = df_rb['consensus_points'].iloc[wr_rb_starters:(wr_rb_starters + wr_rb_bench)].quantile(.9)
qb_replacement_value = df_qb['consensus_points'].iloc[qb_te_starters:(qb_te_starters + qb_te_bench)].quantile(.9)
te_replacement_value = df_te['consensus_points'].iloc[qb_te_starters:(qb_te_starters + qb_te_bench)].quantile(.9)

In [56]:
## calculate cost per point value (budget / ((max points - replacement points) * .95) 
## use 95th percentile to help adjust for outliers. Or what about a weighted percentile? 

wr_cpp = wr_rb_budget / (df_wr['consensus_points'].max() - wr_replacement_value)
rb_cpp = wr_rb_budget / (df_rb['consensus_points'].max() - rb_replacement_value)
qb_cpp = qb_te_budget / (df_qb['consensus_points'].max() - qb_replacement_value)
te_cpp = qb_te_budget / (df_te['consensus_points'].iloc[1] - te_replacement_value)  ## **Manually adjusting for Kelce


In [57]:
## calculate auction value (projection - replacement value) * cost per point

df_wr['auction_value'] = (df_wr['consensus_points'] - wr_replacement_value) * wr_cpp
df_rb['auction_value'] = (df_rb['consensus_points'] - rb_replacement_value) * rb_cpp
df_qb['auction_value'] = (df_qb['consensus_points'] - qb_replacement_value) * qb_cpp
df_te['auction_value'] = (df_te['consensus_points'] - te_replacement_value) * te_cpp


/Users/cameronfryzel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/cameronfryzel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/cameronfryzel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is tryi

In [58]:
df_wr.iloc[0:36]

,name,consensus_points,ds_points,position,auction_value
0,Justin Jefferson,207,284.20,WR,69.000000
1,Ja'Marr Chase,200,266.85,WR,63.194712
2,Tyreek Hill,187,262.15,WR,52.413462
3,Cooper Kupp,186,259.95,WR,51.584135
4,A.J. Brown,186,223.75,WR,51.584135
5,Stefon Diggs,179,246.25,WR,45.778846
6,Davante Adams,177,229.15,WR,44.120192
7,CeeDee Lamb,174,236.30,WR,41.632212
8,Jaylen Waddle,158,202.60,WR,28.362981
9,Amon-Ra St. Brown,153,231.25,WR,24.216346


In [59]:
## repeat for ds point values 
## sort values and reset index
df_wr.sort_values(['ds_points'], ascending = False, inplace = True)
df_wr.reset_index(inplace = True, drop = True)
df_rb.sort_values(['ds_points'], ascending = False, inplace = True)
df_rb.reset_index(inplace = True, drop = True)
df_qb.sort_values(['ds_points'], ascending = False, inplace = True)
df_qb.reset_index(inplace = True, drop = True)
df_te.sort_values(['ds_points'], ascending = False, inplace = True)
df_te.reset_index(inplace = True, drop = True)

## calculate replacement player value (avg of the bench players (aka $1 players/replaceable) 
## UPDATE: 2023 Post Draft Analysis - Everyone goes hard early. Avg of 4.5 players/team with over $10 bids. 
    ## Adjusting replacement player value to 80th percentile of bench players instead of 20th. Won't make a huge
    ## difference but worth adjusting 
wr_replacement_value_ds = df_wr['ds_points'].iloc[wr_rb_starters:(wr_rb_starters + wr_rb_bench)].quantile(.9)
rb_replacement_value_ds = df_rb['ds_points'].iloc[wr_rb_starters:(wr_rb_starters + wr_rb_bench)].quantile(.9)
qb_replacement_value_ds = df_qb['ds_points'].iloc[qb_te_starters:(qb_te_starters + qb_te_bench)].quantile(.9)
te_replacement_value_ds = df_te['ds_points'].iloc[qb_te_starters:(qb_te_starters + qb_te_bench)].quantile(.9)

## calculate cost per point value (budget / ((max points - replacement points) * .95) 
## use 95th percentile to help adjust for outliers. Or what about a weighted percentile? 

wr_cpp_ds = wr_rb_budget / (df_wr['ds_points'].max() - wr_replacement_value_ds)
rb_cpp_ds = wr_rb_budget / (df_rb['ds_points'].max() - rb_replacement_value_ds)
qb_cpp_ds = qb_te_budget / (df_qb['ds_points'].iloc[1] - qb_replacement_value_ds) ## ** Manually adjusting for Allen
te_cpp_ds = qb_te_budget / (df_te['ds_points'].iloc[1] - te_replacement_value_ds) ##** manually  adjusting for Kelce

## based on elboberto method - surplus value at an aggregate level

df_wr['surplus_ds'] = df_wr['ds_points'] - wr_replacement_value_ds
df_rb['surplus_ds'] = df_rb['ds_points'] - rb_replacement_value_ds
df_qb['surplus_ds'] = df_qb['ds_points'] - qb_replacement_value_ds
df_te['surplus_ds'] = df_te['ds_points'] - te_replacement_value_ds

## calculate auction value (projection - replacement value) * cost per point

df_wr['auction_value_ds'] = (df_wr['ds_points'] - wr_replacement_value_ds) * wr_cpp_ds
df_rb['auction_value_ds'] = (df_rb['ds_points'] - rb_replacement_value_ds) * rb_cpp_ds
df_qb['auction_value_ds'] = (df_qb['ds_points'] - qb_replacement_value_ds) * qb_cpp_ds
df_te['auction_value_ds'] = (df_te['ds_points'] - te_replacement_value_ds) * te_cpp_ds

/Users/cameronfryzel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/cameronfryzel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/cameronfryzel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [60]:
## union data frames back together

df_final = pd.concat([df_wr, df_rb, df_qb, df_te])

In [61]:
df_final.reset_index(inplace = True)

In [62]:
df_final['auction_value'] = np.where(df_final['auction_value'] <= 1 ,
                       1,
                       df_final['auction_value'])


In [63]:
df_final['auction_value_ds'] = np.where(df_final['auction_value_ds'] <= 1 ,
                       1,
                       df_final['auction_value_ds'])

In [64]:
df_final.loc[df_final['position'] == 'QB']

##df_final

,index,name,consensus_points,ds_points,position,auction_value,surplus_ds,auction_value_ds
281,0,Josh Allen,379,415.7,QB,26.112360,126.55,41.564809
282,1,Patrick Mahomes,378,374.4,QB,25.797753,85.25,28.000000
283,2,Jalen Hurts,385,360.8,QB,28.000000,71.65,23.533138
284,3,Lamar Jackson,363,356.5,QB,21.078652,67.35,22.120821
285,4,Justin Herbert,326,342.5,QB,9.438202,53.35,17.522581
...,...,...,...,...,...,...,...,...
337,56,Dorian Thompson-Robinson,8,0.0,QB,1.000000,-289.15,1.000000
338,57,Jake Haener,6,0.0,QB,1.000000,-289.15,1.000000
339,58,Tanner McKee,4,0.0,QB,1.000000,-289.15,1.000000
340,59,Malik Cunningham,1,0.0,QB,1.000000,-289.15,1.000000


In [65]:
df_final['surplus_ds'] = np.where(df_final['surplus_ds'] <= 0 ,
                       0,
                       df_final['surplus_ds'])

In [66]:
## surplus

cpp_v2 = (199 * 12) / df_final['surplus_ds'].sum()


In [67]:
df_final['surplus_auction_value'] = df_final['surplus_ds'] * cpp_v2


In [68]:
df_final.loc[df_final['position'] == 'QB'].head(50)

,index,name,consensus_points,ds_points,position,auction_value,surplus_ds,auction_value_ds,surplus_auction_value
281,0,Josh Allen,379,415.7,QB,26.112360,126.55,41.564809,72.861400
282,1,Patrick Mahomes,378,374.4,QB,25.797753,85.25,28.000000,49.082848
283,2,Jalen Hurts,385,360.8,QB,28.000000,71.65,23.533138,41.252622
284,3,Lamar Jackson,363,356.5,QB,21.078652,67.35,22.120821,38.776889
285,4,Justin Herbert,326,342.5,QB,9.438202,53.35,17.522581,30.716363
286,5,Joe Burrow,342,341.5,QB,14.471910,52.35,17.194135,30.140611
287,6,Trevor Lawrence,314,318.9,QB,5.662921,29.75,9.771261,17.128618
288,7,Deshaun Watson,313,318.0,QB,5.348315,28.85,9.475660,16.610442
289,8,Justin Fields,338,311.2,QB,13.213483,22.05,7.242229,12.695329
290,9,Kirk Cousins,294,307.1,QB,1.000000,17.95,5.895601,10.334746


In [69]:
## Set up for objective function 
## convert to dictionaries (data sharks first) 

##create empty dictionaries
values = {}
points = {}
##iterate through unique positions
for pos in df_final.position.unique():
    ## create temp df of each position
    available_pos = df_final[df_final.position == pos]
    ## create k/v pairs for player and auction value
    value = list(available_pos[["name","auction_value_ds"]].set_index("name").to_dict().values())[0]
    point = list(available_pos[["name","ds_points"]].set_index("name").to_dict().values())[0]
    ## insert into dicts
    values[pos] = value
    points[pos] = point

In [70]:
points

{'WR': {' Justin Jefferson': 284.2,
  " Ja'Marr Chase": 266.85,
  ' Tyreek Hill': 262.15,
  ' Cooper Kupp': 259.95,
  ' Stefon Diggs': 246.25,
  ' CeeDee Lamb': 236.3,
  ' Amon-Ra St. Brown': 231.25,
  ' Davante Adams': 229.15,
  ' A.J. Brown': 223.75,
  ' Garrett Wilson': 212.0,
  ' Chris Olave': 203.8,
  ' Jaylen Waddle': 202.6,
  ' Tee Higgins': 201.55,
  ' D.K. Metcalf': 197.4,
  ' Calvin Ridley': 192.95,
  ' DeVonta Smith': 189.85,
  ' Keenan Allen': 187.65,
  ' Deebo Samuel': 187.1,
  ' Tyler Lockett': 185.75,
  ' Amari Cooper': 185.65,
  ' Mike Williams': 185.3,
  ' Christian Watson': 179.8,
  ' Jerry Jeudy': 179.65,
  ' Drake London': 174.95,
  ' Terry McLaurin': 173.8,
  ' Chris Godwin': 171.0,
  ' Christian Kirk': 170.9,
  ' D.J. Moore': 169.5,
  ' Mike Evans': 162.95,
  ' DeAndre Hopkins': 162.6,
  ' Jordan Addison': 162.5,
  ' Brandon Aiyuk': 161.1,
  ' Gabriel Davis': 158.95,
  ' Marquise Brown': 158.3,
  ' Michael Pittman': 158.3,
  ' Diontae Johnson': 155.0,
  ' Jahan Do

In [71]:
## set constraints 

pos_min_wrflex = {
    "QB": 1,
    "RB": 2,
    "WR": 3,
    "TE": 1

}

pos_min_rbflex = {
    "QB": 1,
    "RB": 3,
    "WR": 2,
    "TE": 1

}

auction_dollars = 194.00

In [72]:
## idk what this is doing (I think creating variables for each k/v pair? so WR_Justin_Jefferson becomes a variable to
## get a 1 or 0 binary?)
_vars = {k: pulp.LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

In [73]:
## Let's solve!

## name our problem MaxFF and state we want to maximize something (points)
prob = pulp.LpProblem("Fantasy", pulp.LpMaximize)

#set the empty lists
max_points = []
costs = []
position_constraints = []


for k, v in _vars.items():
    ## sum of player costs
    costs += pulp.lpSum([values[k][i] * _vars[k][i] for i in v])
    ## sum of points
    max_points += pulp.lpSum([points[k][i] * _vars[k][i] for i in v])
    ## constraints on players
    prob += pulp.lpSum([_vars[k][i] for i in v]) <= pos_min_rbflex[k]
    ##prob += pulp.lpSum([pos_flex[k] * _vars[k][i] for i in v]) <= pos_flex_available

## maximize points and keep costs below auction dollars
prob += pulp.lpSum(max_points)
prob += pulp.lpSum(costs) <= auction_dollars

In [74]:
prob.solve()

1

In [75]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))


In [76]:
## Set up for objective function 
## convert to dictionaries (consensus) 

##create empty dictionaries
values_c = {}
points_c = {}
##iterate through unique positions
for pos in df_final.position.unique():
    ## create temp df of each position
    available_pos_c = df_final[df_final.position == pos]
    ## create k/v pairs for player and auction value
    value_c = list(available_pos_c[["name","auction_value"]].set_index("name").to_dict().values())[0]
    point_c = list(available_pos_c[["name","consensus_points"]].set_index("name").to_dict().values())[0]
    ## insert into dicts
    values_c[pos] = value_c
    points_c[pos] = point_c
    
## idk what this is doing (I think creating variables for each k/v pair? so WR_Justin_Jefferson becomes a variable to
## get a 1 or 0 binary?)
_vars_c = {k: pulp.LpVariable.dict(k, v, cat="Binary") for k, v in points_c.items()}



In [77]:
## Let's solve!

## name our problem MaxFF and state we want to maximize something (points)
prob_c = pulp.LpProblem("Fantasy", pulp.LpMaximize)

#set the empty lists
max_points_c = []
costs_c = []
position_constraints_c = []


for k, v in _vars_c.items():
    ## sum of player costs
    costs_c += pulp.lpSum([values_c[k][i] * _vars_c[k][i] for i in v])
    ## sum of points
    max_points_c += pulp.lpSum([points_c[k][i] * _vars_c[k][i] for i in v])
    ## constraints on players
    prob_c += pulp.lpSum([_vars_c[k][i] for i in v]) <= pos_min_rbflex[k]

## maximize points and keep costs below auction dollars
prob_c += pulp.lpSum(max_points_c)
prob_c += pulp.lpSum(costs_c) <= auction_dollars

In [78]:
prob_c.solve()

1

In [79]:
summary(prob_c)

Variables:

QB__Jalen_Hurts = 1.0
RB__Austin_Ekeler = 1.0
RB__D'Andre_Swift = 1.0
RB__Jonathan_Taylor = 1.0
TE__Travis_Kelce = 1.0
WR__Christian_Kirk = 1.0
WR__Jahan_Dotson = 1.0
---------------------------------------

Constraints:
28.0*1.0 + 63.94240837696335*1.0 + 1.8062827225130889*1.0 + 53.10471204188482*1.0 + 44.95412844036697*1.0 = 191.80753158172826
---------------------------------------

Score:
385*1.0 + 221*1.0 + 135*1.0 + 206*1.0 + 168*1.0 + 125*1.0 + 125*1.0 = 1365.0


In [80]:
summary(prob)

Variables:

QB__Josh_Allen = 1.0
RB__A.J._Dillon = 1.0
RB__D'Andre_Swift = 1.0
RB__Khalil_Herbert = 1.0
TE__Travis_Kelce = 1.0
WR__A.J._Brown = 1.0
WR__Justin_Jefferson = 1.0
---------------------------------------

Constraints:
41.56480938416423*1.0 + 5.493311288140702*1.0 + 1.5868029460393913*1.0 + 4.283330828197819*1.0 + 36.25361512791991*1.0 + 35.749970106421145*1.0 + 69.0*1.0 = 193.93183968088317
---------------------------------------

Score:
415.7*1.0 + 152.9*1.0 + 147.25*1.0 + 151.15*1.0 + 226.7*1.0 + 223.75*1.0 + 284.2*1.0 = 1601.65


In [81]:
excel_path = '/Users/cameronfryzel/Documents/FantasyFootball/FF_AuctionDraft_2023.xlsx'

In [82]:
## Export dataframe to excel sheet across multiple sheets
## df_final.loc[df_final['position'] == 'WR']

with pd.ExcelWriter(excel_path) as writer:
    df_final.loc[df_final['position'] == 'QB'].to_excel(writer, sheet_name = 'QB', index = False)
    df_final.loc[df_final['position'] == 'RB'].to_excel(writer, sheet_name = 'RB', index = False)
    df_final.loc[df_final['position'] == 'WR'].to_excel(writer, sheet_name = 'WR', index = False)
    df_final.loc[df_final['position'] == 'TE'].to_excel(writer, sheet_name = 'TE', index = False)